In [1]:
"""
NOTE: Execute this only after USFDA_Diabetes.ipynb is executed and USFDA ingredients are loaded.
"""
from neo4j import GraphDatabase
import os
import pandas as pd
import time
import json
print("Import successful")

Import successful


In [2]:
URI = os.environ["NEO4J_URI"]
USER=os.environ["NEO4J_USER_NAME"]
PASSWORD=os.environ["NEO4J_PASSWD"]
AUTH = (os.environ["NEO4J_USER_NAME"], os.environ["NEO4J_PASSWD"])

In [3]:
#Neo4J connect and Query Boilerplate

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
            
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            #response = (session.run(query, parameters))
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        
        #return pd.DataFrame([r.values() for r in response], columns=response.keys())
        return response
        
    def multi_query(self, multi_line_query, parameters=None, db=None):
        for li in multi_line_query.splitlines():
                print(li)
                result=self.query(li, parameters=None, db=None)
                print(result)

In [4]:
#Make a default connection and it should return `[<Record count(n)=0>]`
conn = Neo4jConnection(uri=URI, 
                       user=USER,
                       pwd=PASSWORD)

#if db is empty, then seed with init values 
res=conn.query('MATCH (n) RETURN count(n)')
print(res)

[<Record count(n)=13776>]


In [ ]:
# NOTE: should be executed only once per database; Alternatively, you can also copy paste this query into neo4j browser
constraint = [
"CREATE CONSTRAINT smk_cat_id FOR (smk_cat:SmokePointCategory) REQUIRE smk_cat.itemID IS UNIQUE;",
"CALL db.awaitIndexes();"]

for cquery in constraint:
    try:
        res = conn.query(cquery)
        print(f"Executed successfully: {cquery}")
    except Exception as e:
        print(f"Error executing query: {cquery}\n{e}")

In [ ]:
load_categories = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/SmokingPoint/nodes.csv') yield map as row", 
"MERGE (i:SmokePointCategory {itemID: row.id}) 
ON CREATE SET i.name = row.smk_category", 
{batchSize:10, iterateList:true, parallel:true})"""


In [13]:
result = conn.query(load_categories)
print("SmokePointCategory nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


SmokePointCategory nodes loaded. Time taken:0 seconds. Committed Operations: 5. Failed Operations:0


In [ ]:
load_rels="""
CALL apoc.periodic.iterate("CALL apoc.load.csv('/SmokingPoint/relationships.csv') yield map as row", 
"MATCH (ing:Ingredient {itemID: row.id1})
MATCH (smk:SmokePointCategory {itemID: row.id2})
MERGE (ing)-[rel:hasSmokePoint]->(smk)
SET rel.main_fat = row.main_fat_type, 
    rel.smoke_point = row.smoke_point, 
    rel.unit = 'F',
    rel.src_ing_name = row.src_ing_name,
    rel.src_url = row.src_url,
    rel.additional_info = row.additional_info", 
{batchSize:10000, iterateList:true, parallel:true})
"""

In [17]:
result = conn.query(load_rels)
print("Relationships loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


Relationships loaded. Time taken:0 seconds. Committed Operations: 65. Failed Operations:0
